<a href="https://colab.research.google.com/github/lllFaNToMlll/NLP-course-work/blob/master/NLPCWRussianNewsLAST_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [ ]:
!git clone https://github.com/IlyaGusev/purano

fatal: destination path 'purano' already exists and is not an empty directory.


In [ ]:
%cd purano

/content/purano


In [ ]:
!pip install -r requirements.txt
!pip install catboost
!pip install tensorflow-text tensorflow-hub
!pip install --upgrade transformers

  Using cached https://files.pythonhosted.org/packages/2c/4e/4f1ede0fd7a36278844a277f8d53c21f88f37f3754abf76a5d6224f76d4a/transformers-3.4.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/35/e7/edf655ae34925aeaefb7b7fcc3dd0887d2a1203ee6b0df4d1170d1a19d4f/tokenizers-0.9.2-cp37-cp37m-manylinux1_x86_64.whl
  Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Found existing installation: transformers 4.6.1
    Uninstalling transformers-4.6.1:
      Successfully uninstalled transformers-4.6.1
  Using cached https://files.pythonhosted.org/packages/d5/43/cfe4ee779bbd6a678ac6a97c5a5cdeb03c35f9eaebbb9720b036680f9a2d/transformers-4.6.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d4/e2/df3543e8ffdab68f5acc73f613de9c2b155ac47f162e725dcac87c521c11/tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
  Fo

In [ ]:
!mkdir models/lang_detect
!mkdir models/cat_detect
!wget https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz -O models/lang_detect/lang_detect_v10.ftz
!wget https://www.dropbox.com/s/23x35wuet280eh6/ru_cat_v5.ftz -O models/cat_detect/ru_cat_v5.ftz

mkdir: cannot create directory ‘models/lang_detect’: File exists
mkdir: cannot create directory ‘models/cat_detect’: File exists
--2021-05-31 19:01:18--  https://www.dropbox.com/s/hoapmnvqlknmu6v/lang_detect_v10.ftz
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz [following]
--2021-05-31 19:01:18--  https://www.dropbox.com/s/raw/hoapmnvqlknmu6v/lang_detect_v10.ftz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc869eba773e6afbb8fe2ba9c830.dl.dropboxusercontent.com/cd/0/inline/BPiCY4qUgCDzolN6zzMh6zgqseB89437hcv7ykL4Aj-KHvyZquoVF2ZeZ_qG72Wjwj8szr72I3EqYgZsIQd_o6oiMuTK_5BBaocC-D8A1MmCmeQuCp4j7onfu5VuSxBEtcXED5TsnfeLjyYqDNAtxAzs/file# [following]
--2021-05-31 19:01:19--  https://uc869e

# Data loading

In [ ]:
# May 25 (training)
!rm -rf data
!mkdir -p data
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz -O - | tar -xz && cd ../
# May 27 and 29 (test)
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0527.tar.gz -O - | tar -xz && cd ../
!cd data && wget https://data-static.usercontent.dev/DataClusteringDataset0529.tar.gz -O - | tar -xz && cd ../

--2021-05-31 19:01:38--  https://data-static.usercontent.dev/DataClusteringDataset0525.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 145541247 (139M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[===================>] 138.80M  14.9MB/s    in 10s     

2021-05-31 19:01:49 (13.9 MB/s) - written to stdout [145541247/145541247]

--2021-05-31 19:01:49--  https://data-static.usercontent.dev/DataClusteringDataset0527.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 166313405 (159M) [application/octet-stream]
Saving to: ‘STDOUT’

-                   100%[=======

In [ ]:
%%writefile configs/cleaner.jsonnet
{
    "lang_detect_model_path": "models/lang_detect/lang_detect_v10.ftz",
    "cat_detect_model_path": "models/cat_detect/ru_cat_v5.ftz",
    "is_lower": true,
    "is_russian_only": true,
    "is_news_only": true
}

Overwriting configs/cleaner.jsonnet


In [ ]:
!rm -f 0525_parsed.db
!python3 -m purano.run_parse --inputs "data/20200525" --output-file 0525_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet
!rm -f 0527_parsed.db
!python3 -m purano.run_parse --inputs "data/20200527" --output-file 0527_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet
!rm -f 0529_parsed.db
!python3 -m purano.run_parse --inputs "data/20200529" --output-file 0529_parsed.db --fmt html --cleaner-config configs/cleaner.jsonnet

Parsed 10000 documents
Parsed 20000 documents
Parsed 30000 documents
Parsed 40000 documents
Parsed 50000 documents
Parsed 60000 documents
Parsed 70000 documents
Parsed 80000 documents
19380 will be saved
<class 'pandas.core.frame.DataFrame'>
Int64Index: 19380 entries, 19379 to 16008
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype                    
---  ------         --------------  -----                    
 0   url            19380 non-null  object                   
 1   host           19380 non-null  object                   
 2   title          19380 non-null  object                   
 3   text           19380 non-null  object                   
 4   date           19380 non-null  datetime64[ns, tzlocal()]
 5   patched_title  19380 non-null  object                   
 6   patched_text   19380 non-null  object                   
 7   category       19380 non-null  object                   
dtypes: datetime64[ns, tzlocal()](1), object(7)
memory usage: 1.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from purano.models import Document

def parse_db(file_name):
    db_engine = "sqlite:///{}".format(file_name)
    engine = create_engine(db_engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    query = session.query(Document)
    docs = query.all()
    records = []
    for doc in docs:
        records.append({
            "title": doc.title,
            "text": doc.text,
            "url": doc.url,
            "host": doc.host,
            "timestamp": doc.date,
            "patched_title": doc.patched_title,
            "patched_text": doc.patched_text
        })
    return records

train_records = parse_db("0525_parsed.db")
test_0527_records = parse_db("0527_parsed.db")
test_0529_records = parse_db("0529_parsed.db")
print(len(train_records))
print(len(test_0527_records))
print(len(test_0529_records))
print(train_records[0])

19380
20080
19096
{'title': 'На пляжах Сочи будут камеры хранения и металлодетекторы', 'text': 'В Сочи с первого дня лета начнут работать 65 пляжей санаторно-курортных учреждений. Они будут доступны только для отдыхающих санатория и будут иметь закрытую приморскую территорию. Сейчас на пляжах устанавливают тревожные кнопки, сигнализацию и системы видеонаблюдения, а после их открытия власти намерены следить за тем, как туристы соблюдают социальную дистанцию. "К началу сезона на 82 городских пляжах установят новые источники освещения и камеры хранения. Такие меры предусмотрены и на пляжах санаториев. Кроме того, прорабатывается вопрос организации стационарных металлодетекторов при входе в береговую зону", - сообщили в пресс-службе мэрии Сочи, передает портал Юга.ру. В администрации курорта добавили: несмотря на то, что в Сочи продолжаются карантинные мероприятия, подготовка к курортному сезону идет по плану, сочинцы и гости курорта смогут посещать общественные места, как только в городе 

In [ ]:
# Fasttext model tuned on older documents
!wget https://www.dropbox.com/s/vttjivmmxw7leea/ru_vectors_v3.bin
# Clustering training markup
!wget https://www.dropbox.com/s/8lu6dw8zcrn840j/ru_clustering_0525_urls.tsv
# Clustering test set
!wget https://www.dropbox.com/s/v2lbfft6pgzoeq3/ru_clustering_test_fixed.tsv
# Headline selection training set
!wget https://www.dropbox.com/s/jpcwryaeszqtrf9/titles_markup_0525_urls.tsv
# Headline selection test set
!wget https://www.dropbox.com/s/v3c0q6msy2pwvfm/titles_markup_test.tsv

--2021-05-31 19:05:17--  https://www.dropbox.com/s/vttjivmmxw7leea/ru_vectors_v3.bin
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/vttjivmmxw7leea/ru_vectors_v3.bin [following]
--2021-05-31 19:05:17--  https://www.dropbox.com/s/raw/vttjivmmxw7leea/ru_vectors_v3.bin
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc39c6c745b07c67d4ec0b6e84ca.dl.dropboxusercontent.com/cd/0/inline/BPhQLvZqU5MYm7j7DaHMQ3nlj1z0XG58YW4YdP_5qBApuwX8iJz_ufdWQOQyEKi5vzdXlxG63VuV2RG5LFC1IaFX3d2HetzEajRE79HxctnKDHioJVkJcoXWSS-BbXjtYMe9AuWFhckCTgsqkYTovg0g/file# [following]
--2021-05-31 19:05:18--  https://uc39c6c745b07c67d4ec0b6e84ca.dl.dropboxusercontent.com/cd/0/inline/BPhQLvZqU5MYm7j7DaHMQ3nlj1z0XG58YW4YdP_5qBApuwX8iJz_ufdWQOQyEKi5vzdXlxG63Vu

# Fetching texts for pretraining

Optional section, required for TF-IDF and Text-Title models

Full list of archives: https://github.com/IlyaGusev/purano/blob/master/raw_datasets.txt

In [ ]:
%%writefile raw_datasets.txt
https://data-static.usercontent.dev/DataClusteringSample0410.tar.gz
https://data-static.usercontent.dev/DataClusteringSample1117.tar.gz

Overwriting raw_datasets.txt


In [ ]:
%%writefile download_datasets.sh
#!/bin/bash

# Download datasets
rm -rf raw_datasets
mkdir raw_datasets
cd raw_datasets
wget -i ../raw_datasets.txt
for F in *.tar.gz; do
    tar -xzf "$F"
    rm -f "$F"
done
cd ../

Writing download_datasets.sh


In [ ]:
!bash download_datasets.sh

--2021-05-31 19:05:28--  https://data-static.usercontent.dev/DataClusteringSample0410.tar.gz
Resolving data-static.usercontent.dev (data-static.usercontent.dev)... 178.128.255.27
Connecting to data-static.usercontent.dev (data-static.usercontent.dev)|178.128.255.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 918103137 (876M) [application/octet-stream]
Saving to: ‘DataClusteringSample0410.tar.gz’

DataClusteringSampl 100%[===================>] 875.57M  19.4MB/s    in 45s     

2021-05-31 19:06:14 (19.6 MB/s) - ‘DataClusteringSample0410.tar.gz’ saved [918103137/918103137]

--2021-05-31 19:06:14--  https://data-static.usercontent.dev/DataClusteringSample1117.tar.gz
Reusing existing connection to data-static.usercontent.dev:443.
HTTP request sent, awaiting response... 200 OK
Length: 910299179 (868M) [application/octet-stream]
Saving to: ‘DataClusteringSample1117.tar.gz’

DataClusteringSampl 100%[===================>] 868.13M  20.0MB/s    in 44s     

2021-05-3

In [ ]:
from purano.io import read_tg_html_dir
from tqdm.notebook import tqdm

from purano.run_parse import DocumentsCleaner

cleaner = DocumentsCleaner("configs/cleaner.jsonnet")

raw_texts = []
pretrain_records = []
for i, record in tqdm(enumerate(read_tg_html_dir("raw_datasets"))):
    cleaned_document = cleaner(record)
    if not cleaned_document:
        continue
    raw_texts.append(cleaned_document["title"])
    raw_texts.append(cleaned_document["text"])
    pretrain_records.append(cleaned_document)

Parsed 10000 documents
Parsed 20000 documents
Parsed 30000 documents
Parsed 40000 documents
Parsed 50000 documents
Parsed 60000 documents
Parsed 70000 documents
Parsed 80000 documents
Parsed 90000 documents
Parsed 100000 documents
Parsed 110000 documents
Parsed 120000 documents
Parsed 130000 documents
Parsed 140000 documents
Parsed 150000 documents
Parsed 160000 documents
Parsed 170000 documents
Parsed 180000 documents
Parsed 190000 documents
Parsed 200000 documents
Parsed 210000 documents
Parsed 220000 documents
Parsed 230000 documents
Parsed 240000 documents
Parsed 250000 documents
Parsed 260000 documents
Parsed 270000 documents
Parsed 280000 documents
Parsed 290000 documents
Parsed 300000 documents
Parsed 310000 documents
Parsed 320000 documents
Parsed 330000 documents
Parsed 340000 documents
Parsed 350000 documents
Parsed 360000 documents
Parsed 370000 documents
Parsed 380000 documents
Parsed 390000 documents
Parsed 400000 documents
Parsed 410000 documents
Parsed 420000 documents
P

In [ ]:
print(len(pretrain_records))

229601


# Clustering

## Evaluation

In [ ]:
from purano.io import read_markup_tsv

train_markup = read_markup_tsv("ru_clustering_0525_urls.tsv")
test_markup = read_markup_tsv("ru_clustering_test_fixed.tsv")

In [ ]:
from collections import Counter
from statistics import median, mean
from sklearn.cluster import AgglomerativeClustering
from purano.clusterer.metrics import calc_metrics

def get_quality(markup, embeds, records, dist_threshold, print_result=False):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_

    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record

    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
        
    metrics = calc_metrics(markup, url2record, url2label)[0]
    if print_result:
        print()
        print("Accuracy: {:.1f}".format(metrics["accuracy"] * 100.0))
        print("Positives Recall: {:.1f}".format(metrics["1"]["recall"] * 100.0))
        print("Positives Precision: {:.1f}".format(metrics["1"]["precision"] * 100.0))
        print("Positives F1: {:.1f}".format(metrics["1"]["f1-score"] * 100.0))
        print("Distance: ", dist_threshold)
        sizes = list(Counter(labels).values())
        print("Max cluster size: ", max(sizes))
        print("Median cluster size: ", median(sizes))
        print("Avg cluster size: {:.2f}".format(mean(sizes)))
        return
    return metrics["1"]["f1-score"]

In [ ]:
import copy

def form_submission(markup, embeds, records, dist_threshold, dataset_name):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_
    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record
    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
    submission = []
    for record in markup:
        dataset = record["dataset"]
        if dataset != dataset_name:
            continue
        first_url = record["first_url"]
        second_url = record["second_url"]
        r = copy.copy(record)
        r["quality"] = "OK" if url2label[first_url] == url2label[second_url] else "BAD"
        submission.append(r)
    return submission

## USE model

In [ ]:
!pip install tensorflow-text tensorflow-hub

In [ ]:
import tensorflow as tf
import tensorflow_text
import tensorflow_hub as hub

module_url = "https://tfhub.dev/google/universal-sentence-encoder-multilingual/3"
use_model = hub.load(module_url)
print("module %s loaded" % module_url)

module https://tfhub.dev/google/universal-sentence-encoder-multilingual/3 loaded


In [ ]:
import numpy as np
import tqdm.notebook as tq


def use_get_embedding(text, model):
    return model([text]).numpy()[0]

def use_records_to_embeds(records, model):
    embeddings = np.zeros((len(records), 512))
    for i, record in tq.tqdm(enumerate(records)):
        embeddings[i] = use_get_embedding(record["title"] + " " + record["text"], model)
    return embeddings


train_use_embeddings = use_records_to_embeds(train_records, use_model)
"""test_0527_use_embeddings = use_records_to_embeds(test_0527_records, use_model)
test_0529_use_embeddings = use_records_to_embeds(test_0529_records, use_model)"""

'test_0527_use_embeddings = use_records_to_embeds(test_0527_records, use_model)\ntest_0529_use_embeddings = use_records_to_embeds(test_0529_records, use_model)'

In [ ]:
get_quality(train_markup, train_use_embeddings, train_records, 0.4, print_result=True)


Accuracy: 89.3
Positives Recall: 92.5
Positives Precision: 86.7
Positives F1: 89.5
Distance:  0.4
Max cluster size:  274
Median cluster size:  1.0
Avg cluster size: 3.31



Accuracy: 89.3
Positives Recall: 92.5
Positives Precision: 86.7
Positives F1: 89.5
Distance:  0.4
Max cluster size:  274
Median cluster size:  1.0
Avg cluster size: 3.31

In [ ]:
"""from purano.io import write_markup_tsv
submission_0527_use = form_submission(test_markup, test_0527_use_embeddings, test_0527_records, 0.37, "0527")
submission_0529_use = form_submission(test_markup, test_0529_use_embeddings, test_0529_records, 0.37, "0529")
full_submission_use = submission_0527_use + submission_0529_use
write_markup_tsv(full_submission_use, "answer.txt", res_key="quality")
!rm -f submission_use.zip
!zip submission_use.zip answer.txt"""

'from purano.io import write_markup_tsv\nsubmission_0527_use = form_submission(test_markup, test_0527_use_embeddings, test_0527_records, 0.37, "0527")\nsubmission_0529_use = form_submission(test_markup, test_0529_use_embeddings, test_0529_records, 0.37, "0529")\nfull_submission_use = submission_0527_use + submission_0529_use\nwrite_markup_tsv(full_submission_use, "answer.txt", res_key="quality")\n!rm -f submission_use.zip\n!zip submission_use.zip answer.txt'

## BERT model

In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
import numpy as np

def bert_get_embedding(text, model):
    return  model.encode([text])[0] #model([text]).numpy()[0] #embedder.encode([text]).numpy()[0]

def bert_records_to_embeds(records, model):
    embeddings = np.zeros((len(records), 768))
    for i, record in tq.tqdm(enumerate(records)):
        embeddings[i] = bert_get_embedding(record["title"] + " " + record["text"], model)
    return embeddings

bert_model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking') #bert-base-nli-stsb-mean-tokens #bert-base-nli-mean-tokens #bert-base-multilingual-cased #distilbert-multilingual-nli-stsb-quora-ranking

train_bert_embeddings = bert_records_to_embeds(train_records, bert_model)
#test_0527_bert_embeddings = bert_records_to_embeds(test_0527_records, bert_model)
#test_0529_bert_embeddings = bert_records_to_embeds(test_0529_records, bert_model)

In [ ]:
def get_quality_bert(markup, embeds, records, dist_threshold, print_result=False):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_

    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record

    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
        
    metrics = calc_metrics(markup, url2record, url2label)[0]
    if print_result:
        print()
        print("Accuracy: {:.1f}".format(metrics["accuracy"] * 100.0))
        print("Positives Recall: {:.1f}".format(metrics["1"]["recall"] * 100.0))
        print("Positives Precision: {:.1f}".format(metrics["1"]["precision"] * 100.0))
        print("Positives F1: {:.1f}".format(metrics["1"]["f1-score"] * 100.0))
        print("Distance: ", dist_threshold)
        sizes = list(Counter(labels).values())
        print("Max cluster size: ", max(sizes))
        print("Median cluster size: ", median(sizes))
        print("Avg cluster size: {:.2f}".format(mean(sizes)))
        return
    return metrics["1"]["f1-score"]

In [ ]:
get_quality_bert(train_markup, train_bert_embeddings, train_records, 0.05, print_result=True) #0.05 distilbert-multilingual-nli-stsb-quora-ranking


Accuracy: 84.1
Positives Recall: 83.5
Positives Precision: 84.0
Positives F1: 83.8
Distance:  0.05
Max cluster size:  893
Median cluster size:  2.0
Avg cluster size: 4.94


In [ ]:
"""from purano.io import write_markup_tsv
submission_0527_bert = form_submission(test_markup, test_0527_bert_embeddings, test_0527_records, 0.37, "0527")
submission_0529_bert = form_submission(test_markup, test_0529_bert_embeddings, test_0529_records, 0.37, "0529")
full_submission_bert = submission_0527_bert + submission_0529_bert
write_markup_tsv(full_submission_bert, "answer.txt", res_key="quality")
!rm -f submission_bert.zip
!zip submission_bert.zip answer.txt"""

'from purano.io import write_markup_tsv\nsubmission_0527_bert = form_submission(test_markup, test_0527_bert_embeddings, test_0527_records, 0.37, "0527")\nsubmission_0529_bert = form_submission(test_markup, test_0529_bert_embeddings, test_0529_records, 0.37, "0529")\nfull_submission_bert = submission_0527_bert + submission_0529_bert\nwrite_markup_tsv(full_submission_bert, "answer.txt", res_key="quality")\n!rm -f submission_bert.zip\n!zip submission_bert.zip answer.txt'

In [ ]:
def get_quality_bert_and_use(markup, embeds, records, dist_threshold, print_result=False):
    clustering_model = AgglomerativeClustering(
        n_clusters=None,
        distance_threshold=dist_threshold,
        linkage="average",
        affinity="cosine"
    )

    clustering_model.fit(embeds)
    labels = clustering_model.labels_

    idx2url = dict()
    url2record = dict()
    for i, record in enumerate(records):
        idx2url[i] = record["url"]
        url2record[record["url"]] = record

    url2label = dict()
    for i, label in enumerate(labels):
        url2label[idx2url[i]] = label
        
    metrics = calc_metrics(markup, url2record, url2label)[0]
    if print_result:
        print()
        print("Accuracy: {:.1f}".format(metrics["accuracy"] * 100.0))
        print("Positives Recall: {:.1f}".format(metrics["1"]["recall"] * 100.0))
        print("Positives Precision: {:.1f}".format(metrics["1"]["precision"] * 100.0))
        print("Positives F1: {:.1f}".format(metrics["1"]["f1-score"] * 100.0))
        print("Distance: ", dist_threshold)
        sizes = list(Counter(labels).values())
        print("Max cluster size: ", max(sizes))
        print("Median cluster size: ", median(sizes))
        print("Avg cluster size: {:.2f}".format(mean(sizes)))
        return
    return metrics["1"]["f1-score"]

In [ ]:
train_bert_and_use_embeddings = np.concatenate((train_bert_embeddings, train_use_embeddings), axis=1)

In [ ]:
train_bert_and_use_embeddings.shape

(19380, 1280)

In [ ]:
buf = np.linspace(0.01,0.10,15)
buf

array([0.01      , 0.01642857, 0.02285714, 0.02928571, 0.03571429,
       0.04214286, 0.04857143, 0.055     , 0.06142857, 0.06785714,
       0.07428571, 0.08071429, 0.08714286, 0.09357143, 0.1       ])

In [33]:
for i in buf:
  get_quality_bert_and_use(train_markup, train_bert_and_use_embeddings, train_records, i, print_result=True) #0.04857142857142858


Accuracy: 56.9
Positives Recall: 12.5
Positives Precision: 99.8
Positives F1: 22.2
Distance:  0.01
Max cluster size:  33
Median cluster size:  1
Avg cluster size: 1.13

Accuracy: 65.4
Positives Recall: 30.2
Positives Precision: 98.4
Positives F1: 46.2
Distance:  0.01642857142857143
Max cluster size:  58
Median cluster size:  1
Avg cluster size: 1.34

Accuracy: 73.7
Positives Recall: 48.1
Positives Precision: 96.9
Positives F1: 64.3
Distance:  0.02285714285714286
Max cluster size:  93
Median cluster size:  1
Avg cluster size: 1.62

Accuracy: 80.3
Positives Recall: 63.4
Positives Precision: 95.0
Positives F1: 76.0
Distance:  0.02928571428571429
Max cluster size:  141
Median cluster size:  1.0
Avg cluster size: 1.96

Accuracy: 84.2
Positives Recall: 74.4
Positives Precision: 92.0
Positives F1: 82.3
Distance:  0.03571428571428572
Max cluster size:  344
Median cluster size:  1.0
Avg cluster size: 2.39

Accuracy: 85.4
Positives Recall: 80.0
Positives Precision: 89.3
Positives F1: 84.4
Dista

In [34]:
train_use_and_bert_embeddings = np.concatenate((train_use_embeddings, train_bert_embeddings), axis=1)

In [35]:
buf = np.linspace(0.1,0.5,15)
buf

array([0.1       , 0.12857143, 0.15714286, 0.18571429, 0.21428571,
       0.24285714, 0.27142857, 0.3       , 0.32857143, 0.35714286,
       0.38571429, 0.41428571, 0.44285714, 0.47142857, 0.5       ])

In [37]:
get_quality_bert_and_use(train_markup, train_use_and_bert_embeddings, train_records, 0.0485, print_result=True)


Accuracy: 85.5
Positives Recall: 83.5
Positives Precision: 86.6
Positives F1: 85.0
Distance:  0.0485
Max cluster size:  809
Median cluster size:  2.0
Avg cluster size: 3.77


Accuracy: 89.3

Positives Recall: 92.5

Positives Precision: 86.7

Positives F1: 89.5

In [36]:
for i in buf:
  get_quality_bert_and_use(train_markup, train_use_and_bert_embeddings, train_records, i, print_result=True)


Accuracy: 75.1
Positives Recall: 93.0
Positives Precision: 68.0
Positives F1: 78.6
Distance:  0.1
Max cluster size:  3295
Median cluster size:  4
Avg cluster size: 34.55

Accuracy: 67.1
Positives Recall: 95.0
Positives Precision: 60.5
Positives F1: 74.0
Distance:  0.1285714285714286
Max cluster size:  7109
Median cluster size:  5.0
Avg cluster size: 118.17

Accuracy: 54.7
Positives Recall: 99.5
Positives Precision: 52.0
Positives F1: 68.4
Distance:  0.15714285714285714
Max cluster size:  16914
Median cluster size:  4.0
Avg cluster size: 358.89

Accuracy: 49.4
Positives Recall: 100.0
Positives Precision: 49.3
Positives F1: 66.0
Distance:  0.18571428571428572
Max cluster size:  19318
Median cluster size:  2.0
Avg cluster size: 969.00

Accuracy: 49.2
Positives Recall: 100.0
Positives Precision: 49.2
Positives F1: 66.0
Distance:  0.2142857142857143
Max cluster size:  19372
Median cluster size:  1
Avg cluster size: 2768.57

Accuracy: 49.2
Positives Recall: 100.0
Positives Precision: 49.2
P

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Accuracy: 49.2
Positives Recall: 100.0
Positives Precision: 49.2
Positives F1: 66.0
Distance:  0.27142857142857146
Max cluster size:  19380
Median cluster size:  19380
Avg cluster size: 19380.00


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))



Accuracy: 49.2
Positives Recall: 100.0
Positives Precision: 49.2
Positives F1: 66.0
Distance:  0.30000000000000004
Max cluster size:  19380
Median cluster size:  19380
Avg cluster size: 19380.00


KeyboardInterrupt: ignored